In [ ]:
import numpy as np
import folium
from sklearn import svm

import warnings
warnings.filterwarnings('ignore')

In [ ]:
import requests
import numpy as np
import pickle
import base64

server_url = "https://8094-01j03cpsnxgzf6zqczh86561ch.cloudspaces.litng.ai/predict"


def get_embeddings(lat, lon, start_date, end_date):
    """
    Get embeddings from the hosted model provided by GreenLens
    """
    response = requests.post(
        server_url,
        json={
            "lat": lat,
            "lon":  lon,
            "startdate": start_date,
            "enddate": end_date,
            "size": 256,  # embeddings can take any size from 8-256
            "whoami": "yc-collab",
        },
    )
    if response.status_code == 200:
        base64_encoded_array = response.json()["output"]
        pickled_array = base64.b64decode(base64_encoded_array)
        embeddings = pickle.loads(pickled_array)
        return embeddings
    elif response.status_code == 401:
        raise ValueError(response.content)
    else:
        print(
            "Failed to get response from the server, status code:", response.status_code
        )

def get_embeddings_many(latlons, start_date, end_date):
    return [get_embeddings(lat, lon, start_date, end_date) for lat, lon in latlons]


In [ ]:
points_true = [  # airports
  (40.776824, -73.873489),
  (41.727228, -71.431166),
  (40.696258, -74.172668),
  (42.364862, -71.015806),
  (38.853499, -77.039526),
  (35.877937, -78.793928),
]

points_false = [  # not airpots
  (31.743262, -87.594122),
  (32.188877, -92.428149),
  (31.978015, -102.747742),
  (32.769798, -106.904086),
  (34.426534, -119.705648),
  (33.810461, -118.287952),
  (37.760997, -122.426817),
]

points_test = [
  (42.848886, -79.962231),
  (41.862865, -80.894946),
  (42.580044, -80.468506),
  (41.411103, -81.842124),  # airport
  (43.630819, -79.394762),  # airport
]


In [ ]:
embeddings_true = get_embeddings_many(points_true, "2023-01-01", "2023-01-10")
embeddings_false = get_embeddings_many(points_false, "2023-06-01", "2023-10-30")

embeddings_test = get_embeddings_many(points_test, "2023-06-01", "2023-09-30")

In [ ]:
x = [i.flatten() for i in embeddings_true + embeddings_false]
y = [1] * len(embeddings_true) + [0] * len(embeddings_false)

z = [i.flatten() for i in embeddings_test]

clf = svm.SVC()
clf.fit(x, y)

In [ ]:
prediction = clf.predict(z)
negative_points = [points_test[index] for index, val in enumerate(prediction) if val == 0]
positive_points = [points_test[index] for index, val in enumerate(prediction) if val == 1]

In [ ]:
def plot_points_on_map(positive_points, negative_points):
    """
    Plots a list of points on a map using folium.
    
    Parameters:
    points (list of tuples): A list of (latitude, longitude) points.
    
    Returns:
    folium.Map: The folium Map object with the plotted points.
    """
    points = positive_points + negative_points
    # Check if the points list is empty
    if not points:
        raise ValueError("The points list is empty.")

    # Calculate the average latitude and longitude to center the map
    avg_lat = sum(lat for lat, lon in points) / len(points)
    avg_lon = sum(lon for lat, lon in points) / len(points)
    
    # Create a folium map centered around the average coordinates
    map_center = [avg_lat, avg_lon]
    folium_map = folium.Map(location=map_center, zoom_start=8)

    # Add points to the map
    for lat, lon in positive_points:
        folium.Marker(location=[lat, lon], icon=folium.Icon(color="blue")).add_to(folium_map)
    for lat, lon in negative_points:
        folium.Marker(location=[lat, lon], icon=folium.Icon(color="red")).add_to(folium_map)
    
    return folium_map

In [ ]:
plot_points_on_map(positive_points, negative_points)